In [ ]:
# Implement code for configuring GPU and parallel processing for images
# Implement code for creating testing, validation and training dataset using the
# flow_from_dataframe API from tensorflow

In [ ]:
# !pip install tqdm
# !pip install tensorflow

In [28]:
import pandas as pd
import os
import shutil
from tqdm.notebook import tqdm as tqdm
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense

df = pd.read_csv('D:/temp/train.csv',dtype=str).head(100)
print("Original shape: {}".format(df.shape))
df['filename'] = ["".join([str(df['patient_id'][i]),'_',str(df['image_id'][i]),'.png']) \
                  for i in range(len(df))]
df = df.loc[:,['filename','cancer']]
df.head()

Original shape: (100, 14)


,filename,cancer
0,10006_462822612.png,0
1,10006_1459541791.png,0
2,10006_1864590858.png,0
3,10006_1874946579.png,0
4,10011_220375232.png,0


In [26]:
def load_img(path):
    
    img = tf.io.read_file(path)
    img = tf.image.decode_png(img, channels = 1)
    img = tf.image.resize(img, [IMAGE_SIZE, IMAGE_SIZE])
    img = tf.cast(img, dtype = tf.float32)
    img = img/255.0
    return img

In [29]:
path = 'D:/temp/train_new/'
IMAGE_SIZE=256
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)
training = datagen.flow_from_dataframe(df,
    directory=path,
    x_col='filename',
    y_col='cancer',
    weight_col=None,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    color_mode='rgb',
    class_mode='binary',
    batch_size=10,
    shuffle=True,
    save_format='png',
    subset='training',
    interpolation='nearest',
    validate_filenames=True,
)
# x_train,x_test,y_train,y_test = train_test_split(df.filename,df.cancer,stratify=df.cancer,test=0.20)
# model.fit(datagen.flow(x_train, y_train, batch_size=32,
#          subset='training'),
#          validation_data=datagen.flow(x_train, y_train,
#          batch_size=8, subset='validation'),
#          steps_per_epoch=len(x_train) / 32, epochs=epochs)

Found 100 validated image filenames belonging to 2 classes.


In [30]:
pretrained_model = ResNet50(
    include_top=False,
    weights="imagenet",
    input_shape=(256,256,3),
    pooling="avg",
    classes=2
)
for layer in pretrained_model.layers:
    layer.trainable = False

In [53]:
def build_model():
    x = pretrained_model.output
    x = Flatten()(x)
    x = BatchNormalization()(x)
    x = Dense(512,activation = 'relu')(x)
    x = Dropout(0.25)(x)
    out = Dense(1,activation = 'sigmoid')(x)
    model = tf.keras.Model(inputs = pretrained_model.input,outputs=out)
    return model

In [54]:
model = build_model()

In [55]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 262, 262, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 128, 128, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                            

In [51]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = 3e-4),
                  loss=tf.keras.losses.BinaryCrossentropy(from_logits = False),
                  metrics=['accuracy'])

In [52]:
m = model.fit(training,epochs=10)

Epoch 1/10
10/10 [==============================] - 11s 789ms/step - loss: 0.5538 - accuracy: 0.7300
Epoch 2/10
10/10 [==============================] - 8s 797ms/step - loss: 0.2079 - accuracy: 0.9700
Epoch 3/10
10/10 [==============================] - 8s 795ms/step - loss: 0.1206 - accuracy: 0.9700
Epoch 4/10
10/10 [==============================] - 8s 807ms/step - loss: 0.0737 - accuracy: 0.9900
Epoch 5/10
10/10 [==============================] - 8s 794ms/step - loss: 0.0465 - accuracy: 0.9900
Epoch 6/10
10/10 [==============================] - 8s 798ms/step - loss: 0.0446 - accuracy: 0.9900
Epoch 7/10
10/10 [==============================] - 8s 810ms/step - loss: 0.0349 - accuracy: 0.9800
Epoch 8/10
10/10 [==============================] - 8s 794ms/step - loss: 0.0622 - accuracy: 0.9800
Epoch 9/10
10/10 [==============================] - 8s 800ms/step - loss: 0.0216 - accuracy: 1.0000
Epoch 10/10
10/10 [==============================] - 8s 801ms/step - loss: 0.0222 - accuracy: 1.000